In [2]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

In [4]:
# 테스트 브라우저 열어보기
# https://map.naver.com/p?c=15.00,0,0,0,dh
driver = wb.Chrome()
url = "https://map.naver.com/p?c=15.00,0,0,0,dh"
driver.get(url)

In [8]:
# 이수역 맛집
search = driver.find_element(By.ID, "input_search1763950216170")

In [10]:
search.send_keys("이수역 맛집")
search.send_keys(Keys.ENTER)

In [14]:
# 이수역 맛집 검색 -> 새로운 방법
query = input("검색할 지역의 맛집>> ")

검색할 지역의 맛집>>  홍대 맛집


In [15]:
# 검색란 가져오기 -> class로
search_input = driver.find_element(By.CSS_SELECTOR, ".input_search")

In [16]:
search_input.send_keys(query + '\n')

In [17]:
# 가게명 수집하기
driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li:nth-child(2) > div.CHC5F > div.bSoi3 > a > span.TYaxT")

[]

In [18]:
driver2 = wb.Chrome()
url2 = 'https://pcmap.place.naver.com/place/list?query=%ED%99%8D%EB%8C%80%20%EB%A7%9B%EC%A7%91&x=126.98067960000259&y=37.4863012999992&clientX=127.035639&clientY=37.641403&bounds=126.97653826928484%3B37.480043748316305%3B126.98503550744374%3B37.49238807526318&display=70&ts=1763951075230&additionalHeight=76&locale=ko&mapUrl=https%3A%2F%2Fmap.naver.com%2Fp%2Fsearch%2F%ED%99%8D%EB%8C%80%20%EB%A7%9B%EC%A7%91'
driver2.get(url2)

In [21]:
food_store = driver2.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.CHC5F > div.bSoi3 > a > span.TYaxT")

In [22]:
food_store[0].text

'더기와 합정점'

In [23]:
# iframe 창으로 전환하는 함수
driver.switch_to.frame("searchIframe") # iframe의 id값만 가져오기

In [28]:
# 가게명 수집하기
food_stores = driver.find_elements(By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li > div.CHC5F > div.bSoi3 > a > span.TYaxT")

In [29]:
food_stores[0].text

'더기와 합정점'